#### Gaze estimation using DeepGaze

In [ ]:
from deepgaze.head_pose_estimation import CnnHeadPoseEstimator
import cv2
import numpy as np
import os
import tensorflow as tf

In [ ]:
# Rotation matrix
# Source: https://github.com/mpatacchiola/deepgaze/issues/43
def yaw2rotmat(yaw):
    x = 0.0
    y = 0.0
    z = yaw
    ch = np.cos(z)
    sh = np.sin(z)
    ca = np.cos(y)
    sa = np.sin(y)
    cb = np.cos(x)
    sb = np.sin(x)
    rot = np.zeros((3,3), 'float32')
    rot[0][0] = ch * ca
    rot[0][1] = sh*sb - ch*sa*cb
    rot[0][2] = ch*sa*sb + sh*cb
    rot[1][0] = sa
    rot[1][1] = ca * cb
    rot[1][2] = -ca * sb
    rot[2][0] = -sh * ca
    rot[2][1] = sh*sa*cb + ch*sb
    rot[2][2] = -sh*sa*sb + ch*cb
    return rot

In [ ]:
def estimatorConfig():
  is_cuda_set = os.getenv('CUDA_VISIBLE_DEVICES')
  if is_cuda_set is None:
      os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

  gpu = False
  if tf.test.gpu_device_name():
      print('GPU found')
      gpu = True
  else:
      print("No GPU found")

  config = tf.ConfigProto(allow_soft_placement=True)
  if gpu:
      config.gpu_options.per_process_gpu_memory_fraction = 0.5
  tf_session = tf.Session(config=config) 
  estimator = CnnHeadPoseEstimator(tf_session) 

  model_dir = os.getenv('MODEL_DIR', './models')
  estimator.load_yaw_variables(os.path.join(model_dir, 'headpose', 'yaw', 'cnn_cccdd_30k.tf'))
  estimator.load_pitch_variables(os.path.join(model_dir, 'headpose', 'pitch', 'cnn_cccdd_30k.tf')) 
  estimator.load_roll_variables(os.path.join(model_dir, 'headpose', 'roll', 'cnn_cccdd_30k.tf'))

  return estimator

In [ ]:
def get_head_pose_vector(estimator, image, face):
    c_x = image.shape[1] / 2
    c_y = image.shape[0] / 2
    f_x = c_x / np.tan(60/2 * np.pi / 180)
    f_y = f_x
    camera_matrix = np.float32([[f_x, 0.0, c_x],
                                [0.0, f_y, c_y],
                                [0.0, 0.0, 1.0] ])

    # define axis for footage overlay if desired
    axis = np.float32([[0.0, 0.0, 0.0],
                       [0.0, 0.0, 0.0],
                       [0.0, 0.0, 0.5]])
    # rotation matrix using DeepGaze convention
    rot_matrix = yaw2rotmat(-yaw[0,0,0])

    roll_degree = estimator.return_roll(image, radians=False)  
    pitch_degree = estimator.return_pitch(image, radians=False)  
    yaw_degree = estimator.return_yaw(image, radians=False)  

    roll = np.deg2rad(roll_degree)
    pitch = np.deg2rad(pitch_degree)
    yaw = np.deg2rad(yaw_degree)

    return (roll, pitch, yaw)